In [1]:
import taskingai
# Load TaskingAI API Key from environment variable

# TaskingAI Model Inference: Chat Completion

In [2]:
from taskingai.inference import *
import json
# choose an available chat_completion model from your project
model_id = "Gk1145Bl"

In [3]:
# normal 
chat_completion = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
    ]
)
chat_completion

{'created_timestamp': 1701173269243,
 'finish_reason': 'stop',
 'message': {'content': 'Hello! How can I assist you today?',
             'function_call': None,
             'role': 'assistant'},
 'object': 'ChatCompletion'}

In [4]:
# multi round chat completion
chat_completion = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
        AssistantMessage("Hello! How can I assist you today?"),
        UserMessage("Can you tell me a joke?"),
        AssistantMessage("Sure, here is a joke for you: Why don't scientists trust atoms? Because they make up everything!"),
        UserMessage("That's funny. Can you tell me another one?"),
    ]
)
chat_completion

{'created_timestamp': 1701173272255,
 'finish_reason': 'stop',
 'message': {'content': "Of course! Here's another joke for you: Why don't "
                        "skeletons fight each other? They don't have the guts!",
             'function_call': None,
             'role': 'assistant'},
 'object': 'ChatCompletion'}

In [5]:
# config max tokens
chat_completion = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
        AssistantMessage("Hello! How can I assist you today?"),
        UserMessage("Can you tell me a joke?"),
        AssistantMessage("Sure, here is a joke for you: Why don't scientists trust atoms? Because they make up everything!"),
        UserMessage("That's funny. Can you tell me another one?"),
    ],
    configs={
        "max_tokens": 5
    }
)
chat_completion

{'created_timestamp': 1701173274744,
 'finish_reason': 'length',
 'message': {'content': "Of course! Here's",
             'function_call': None,
             'role': 'assistant'},
 'object': 'ChatCompletion'}

In [6]:
# function call
function = Function(
    name="plus_a_and_b",
    description="Sum up a and b and return the result",
    parameters={
        "type": "object",
        "properties": {
            "a": {
                "type": "integer",
                "description": "The first number"
            },
            "b": {
                "type": "integer",
                "description": "The second number"
            }
        },
        "required": ["a", "b"]
    },
)
chat_completion = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("What is the result of 112 plus 22?"),
    ],
    functions=[function]
)
print(f"chat_completion = {chat_completion}")

assistant_function_call_message = chat_completion.message
fucntion_name = assistant_function_call_message.function_call.name
argument_content = json.dumps(assistant_function_call_message.function_call.arguments)
print(f"function name: {fucntion_name}, argument content: {argument_content}")

chat_completion = {'created_timestamp': 1701173277776,
 'finish_reason': 'function_call',
 'message': {'content': None,
             'function_call': {'arguments': {'a': 112, 'b': 22},
                               'name': 'plus_a_and_b'},
             'role': 'assistant'},
 'object': 'ChatCompletion'}
function name: plus_a_and_b, argument content: {"a": 112, "b": 22}


In [7]:
# add function message
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("What is the result of 112 plus 22?"),
        assistant_function_call_message,
        FunctionMessage(name=fucntion_name, content="144")
    ],
    functions=[function]
)
chat_completion

{'created_timestamp': 1701173282280,
 'finish_reason': 'stop',
 'message': {'content': 'The result of 112 plus 22 is 144.',
             'function_call': None,
             'role': 'assistant'},
 'object': 'ChatCompletion'}